## General setup 

In [1]:
#pip install --upgrade gensim

In [2]:
#import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

NameError: name 'logging' is not defined

In [3]:
import os
import glob
import pandas as pd
import numpy as np
import pickle
import nltk
import graphlab
from graphlab import SFrame

from nltk.collocations import *       #Bad form
from nltk.corpus import stopwords
from nltk.tokenize import *
import sklearn
from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest
from sklearn.decomposition import PCA
from sklearn.pipeline import FeatureUnion
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn import pipeline
from sklearn.grid_search import GridSearchCV
from prettytable import PrettyTable
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import string
#from gensim import interfacesys, time
from gensim import corpora, models, similarities
from gensim.corpora import SvmLightCorpus
from gensim import corpora

from nltk.stem.snowball import SnowballStemmer
#set data path
LOCAL_DATA_PATH = 'C:\Users\Kevin OGallath34\political_history\processed_data'
os.chdir(LOCAL_DATA_PATH)

C:\Users\Kevin OGallath34\Anaconda2\envs\gl-env\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Kevin OGallath34\Anaconda2\envs\gl-env\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


## Read pickle file


In [4]:
#import data
labels = pickle.load(open('bow_labels.pkl', "r"))
text = pickle.load(open('bow_processed_text.pkl', "r"))

In [5]:
candidates_df = pd.read_pickle('C:\Users\Kevin OGallath34\political_history\processed_data\candidates_df1.pkl')

In [ ]:
#from nltk import PorterStemmer
#stemmer = PorterStemmer()

In [6]:
sf_init = ['strength','dominance','authorization','potency','intensity','force','control']
# sf_list manually creatd by reviewing 2 debate texts "
sf_man = ["dominance" ,"authorization" ,"potency", "intensity" ,"force", "control" ,"dominate" ,"strengthen", "authorize" ,"evil" ,"immoral" ,"obey", "character", "moral" ,"crimes" ,"punishment", "prosecute" ,"values","military", "federal", "violence" ,"amnesty" ,"illegally" ,"bad", "disaster", "stop" ,"trouble" ,"dishorned" ,"return" ,"proliferate" ,"elite" ,"illegal" ,"stolen" ,"criminally" ,"criminal" ,"jail" ,"crooked" ,"charges" ,"false" ,"disgrace" ,"lawsuits" ,"corrupt" ,"guilty"," fraud", "racketeering" ,"judgement" ,"legitimize","repressive", "brutal" ,"agressive", "kill" ,"exploit" ,"greed","excess" ,"corruption" ,"evil","runaway" ,"indictment" ,"unwise" ,"bankrupt" ,"consequence" ,"torture","capture","crush","win","poppy", "drug" ,"heroin" ,"failed" ,"fault" ,"defeated" ,"withdrawal","league","execute","sanctions" ,"rogue", "regime" ,"torture" ,"sacrifice" ,"right" ,"wrong" ,"adversaries" ,"military" ,"control" ,"states" ,"local" ,"parents" ,"corporations" ,"rich" ,"criminal" ,"justice" ,"penalty" ,"law" ,"order","police", "amendment", "constitution", "uphold" ,"seige","protecting","defending","defend","status","fathers" ,"again" ,"honored" ,"back" ,"punished" ,"punish" ,"strong" ,"borders" ,"wall" ,"secure" ,"private" ,"powerful" ,"tough","tougher","top" ,"fallen" ,"renegotiate" ,"deals" ,"ideals" ,"isolate", "isolation","trust" ,"verify" ,"oversight" ,"enforce" ,"proud" ,"parental", "powerful" ,"wealthy","negotiate", "leaderships", "leaders" ,"tougher" ,"repeal" ,"strongly", "failure", "responsibility","win" ,"victory" ,"surge" , "stability" ,"respect","restraint" ,"solidarity" ,"rewarded" ,"influence" ,"power" ,"stricter" ,"fundamental" ,"belief" ,"goodness" ,"paternal", "eliminate", "freeze" ,"victory" ,"honor" ,"back","holding" ,"responsibly" ,"bolster" ,"toughest" ,"unwarrented","acceptable","behavior", "violation","sieze" ,"respects" ,"illegal" ,"objectionable","secure","invasion" ,"influence","oversight","accountability"]


In [7]:
stemsR = [] 
#define function to tokenize and stem

def stemsR(text2):
   
# Removes punctuation, converts all characters to lowercase, removes stop words, stems
    tokens = RegexpTokenizer(r'\w+')
    stops = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    
    token = tokens.tokenize(text2)
    filtered_words = [word for word in token if word in sf_man]
    filtered_stop = [word for word in filtered_words if word not in stops]
    stems2 = [stemmer.stem(t) for t in filtered_stop]
    return( " ".join(filtered_stop))

for _, text2 in candidates_df.iteritems():
    candidates_df['stemsR'] =  candidates_df['text2'].apply(lambda row: stemsR(row))
  
    print candidates_df['stemsR']

index
3                                          
6                         moral moral moral
7                                          
8                                          
9                         eliminate respect
10                                         
11                                stop stop
12                                         
14                                     back
15                                         
17                                         
18                                         
19                                         
20                                         
21                                         
22                                         
25                               back right
27                                         
30                                         
32                                         
33                                    right
34                                         
36                        

In [8]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(candidates_df['stemsR'].values)

freq_term_matrix = count_vectorizer.transform(candidates_df['stemsR'])
print count_vectorizer.vocabulary_

{u'violation': 141, u'top': 129, u'stability': 119, u'kill': 69, u'renegotiate': 101, u'protecting': 94, u'rewarded': 110, u'police': 87, u'uphold': 137, u'justice': 68, u'character': 16, u'isolation': 66, u'penalty': 86, u'failed': 42, u'parents': 84, u'rich': 111, u'criminal': 25, u'local': 75, u'prosecute': 93, u'return': 109, u'adversaries': 2, u'stop': 123, u'rogue': 113, u'wealthy': 144, u'trouble': 134, u'execute': 40, u'deals': 29, u'bad': 8, u'victory': 140, u'withdrawal': 146, u'poppy': 88, u'amnesty': 4, u'enforce': 37, u'amendment': 3, u'right': 112, u'force': 49, u'leaders': 72, u'tougher': 132, u'fathers': 46, u'crimes': 24, u'back': 7, u'freeze': 50, u'ideals': 59, u'corporations': 21, u'corrupt': 22, u'belief': 11, u'repressive': 103, u'respects': 105, u'constitution': 19, u'federal': 48, u'defend': 31, u'criminally': 26, u'paternal': 85, u'crush': 28, u'honored': 58, u'eliminate': 36, u'legitimize': 74, u'corruption': 23, u'illegally': 61, u'obey': 79, u'power': 89, u'

In [9]:
features_count = freq_term_matrix.sum(axis=0).tolist()[0]
features_names = count_vectorizer.get_feature_names()
features = pd.DataFrame(zip(features_names, features_count), 
                                columns=['features', 'count']
                               ).sort_values(by=['features'], ascending=False)
 

In [10]:
tfidf = TfidfVectorizer(max_features=10, strip_accents='unicode', analyzer='word', stop_words=None, lowercase=True, use_idf=True)
X = tfidf.fit_transform(candidates_df['stemsR']) # the matrix articles x max_features(=words)
for i, row in enumerate(X):
    print X[i]





  (0, 6)	1.0



  (0, 0)	1.0







  (0, 7)	0.681335425686
  (0, 0)	0.731971336669



  (0, 7)	1.0





  (0, 0)	1.0









  (0, 7)	1.0












  (0, 7)	1.0

  (0, 7)	0.681335425686
  (0, 0)	0.731971336669



  (0, 7)	1.0

  (0, 7)	1.0
  (0, 0)	1.0




  (0, 9)	1.0
  (0, 0)	1.0
  (0, 0)	1.0



  (0, 7)	1.0





  (0, 7)	1.0






  (0, 9)	1.0








  (0, 0)	1.0
  (0, 0)	1.0

  (0, 7)	1.0






















  (0, 1)	1.0
  (0, 0)	1.0
  (0, 0)	1.0



  (0, 7)	1.0
  (0, 4)	1.0

  (0, 9)	1.0
  (0, 0)	1.0


  (0, 0)	1.0














  (0, 7)	1.0









  (0, 5)	1.0


  (0, 7)	1.0

  (0, 0)	1.0






  (0, 7)	1.0
  (0, 9)	0.797788063976
  (0, 7)	0.602937977721
  (0, 3)	1.0
  (0, 3)	0.780571030743
  (0, 0)	0.625067089171
  (0, 3)	1.0




  (0, 7)	0.681335425686
  (0, 0)	0.731971336669









  (0, 7)	1.0


  (0, 4)	1.0
  (0, 7)	1.0
  (0, 7)	1.0




  (0, 7)	1.0



  (0, 7)	1.0











  (0, 7)	1.0







  (0, 7)	1.0

  (0, 3)	1.0


  (0, 0)	1.0



  (0, 0)	1.0









In [11]:
tf_idf_matrix = tfidf.transform(freq_term_matrix)
dense = X.toarray()
print tf_idf_matrix.toarray

AttributeError: lower not found

## NF word processing

In [ ]:
# nf_list manually creatd by reviewing 2 debate texts 
np_init = ('sorrow','nourishment','sympathy','feeling','care','aid','attend','nuture','nourish')
np_man = ['sympathy' , 'nourishment' , 'sorrow' , 'feeling' , 'care' ,'aid' , 'raise', 'attend' , 'nourish', 'community', 'clean', 'consumer',  'education', 'care', 'environmentalism', 'rights', 'arts', 'humanities', 'credits', 'toddlers', 'reasonable', 'sensible', 'common', 'sense', 'choice','benefits', 'life','resources','shared', 'new', 'workers', 'reform','common', 'free', 'open' ,'neighbors','heartbreaking', 'painful', 'personal', 'individual', 'choice','peace', 'thrives', 'grow', 'opportunities', 'change','help', 'equal', 'system', 'community', 'free','public','fair', 'share', 'partnership','agreements', 'reform','future', 'together', 'diversity', 'lift', 'charity', 'partnership', 'help','fair','safe', 'haven', 'up', 'lift', 'concerns', 'better', 'all', 'rising', 'serve', 'need', 'needed', 'needy','support', 'helpful', 'child', 'mother', 'plan', 'raise', 'careful', 'national', 'replenish', 'benefits', 'social', 'act', 'help', 'grow' , 'fairer', 'future', 'proposals','rescue', 'down', 'together', 'under', 'enter', 'fill', 'rescue' , 'global', 'grid', 'subsidy', 'entitlement' , 'entitled', 'change', 'relationship', 'natural', 'resources', 'cooperate', 'broaden', 'global','diplomacy', 'alliances','peace','unified', 'root', 'pesticides', 'hurts', 'poverty','discriminate', 'hate', 'abuse', 'down', 'waste','nuclear', 'chemical'] 
print np_man

In [ ]:
stemsD = [] 
#define function to tokenize and stem

def stemsD(text2):
# Removes punctuation, converts all characters to lowercase, removes stop words, stems
    tokens = RegexpTokenizer(r'\w+')
    stops = set(stopwords.words('english'))
    stemmer = SnowballStemmer('english')
    
    token = tokens.tokenize(text2)
    filtered_words = [word for word in token if word in np_man]
    filtered_stop = [word for word in filtered_words if word not in stops]
    stemsD = [stemmer.stem(t) for t in filtered_stop]
    return( " ".join(filtered_stop))

for _, text2 in candidates_df.iteritems():
    candidates_df['stemsD'] =  candidates_df['text2'].apply(lambda row: stemsD(row))
  


In [ ]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(candidates_df['stemsD'].values)

freq_term_matrix = count_vectorizer.transform(candidates_df['stemsD'])
print count_vectorizer.vocabulary_

In [ ]:
features_count = freq_term_matrix.sum(axis=0).tolist()[0]
features_names = count_vectorizer.get_feature_names()
features = pd.DataFrame(zip(features_names, features_count), 
                                columns=['features', 'count']
                               ).sort_values(by=['features'], ascending=False)
print features_names 

In [ ]:
import re
from collections import Counter


#define function to tokenize and stem

def count1(text2):
    C1 = 0
# Removes punctuation, converts all characters to lowercase, removes stop words, stems
    words = text2.split()
    C1 =  len(words) 
    return(C1)

for _, text2 in candidates_df.iteritems():
    candidates_df['C1'] =  candidates_df['text2'].apply(lambda row: count1(row))
  

In [ ]:
#define function to tokenize and stem

def countD(stemsD):
    D1 = 0
# Removes punctuation, converts all characters to lowercase, removes stop words, stems
    words = stemsD.split()
    D1 += len(words) 
    return(D1)
    
for _, text2 in candidates_df.iteritems():
    candidates_df['D1'] =  candidates_df['stemsD'].apply(lambda row: countD(row)) 

In [ ]:
#define function to tokenize and stem

def countR(stemsR):
    R1 = 0
    # Removes punctuation, converts all characters to lowercase, removes stop words, stems
    words = stemsR.split()
    R1 += len(words) 
    return(R1)

for _, text2 in candidates_df.iteritems():
    candidates_df['R1'] =  candidates_df['stemsR'].apply(lambda row: countR(row))  

In [ ]:
df = candidates_df.loc[:,['affiliation','C1','D1','R1','stemsD','stemsR']]

df.to_csv("C:/Users/Kevin OGallath34/political_history/processed_data/test.csv", index=False, cols=('affiliation','C1','D1','R1','stemsD','stemsR'))

In [ ]:
# export new dataframe for others to use
directory_name = LOCAL_SAVE_PATH
base_filename = 'Topic_Modeling_df'
suffix = '.pkl'
save_path = os.path.join('C:\Users\Kevin OGallath34\political_history\processed_data')
candidates_df.to_pickle(save_path)

## Word2vec

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec.load_word2vec_format('C:\Users\Kevin OGallath34\political_history\processed_data\knowledge-vectors-skipgram1000.bin',
                                       binary=True)

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec.load_word2vec_format('C:\Users\Kevin OGallath34\political_history\processed_data\knowledge-vectors-skipgram1000.bin',
                                       binary=True)
model.save_word2vec_format('C:\Users\Kevin OGallath34\political_history\processed_data\skipgram.txt', binary=False)

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec.load_word2vec_format('C:\Users\Kevin OGallath34\political_history\processed_data\skipgram.txt', binary=False)

In [ ]:
model.most_similar('control')

In [ ]:
model.most_similiar('strength',topN=10)

In [ ]:
model.most_similar('nourishment',topN=10)

In [ ]:
model.most_similar('care',topN=10)

In [ ]:
# -*- coding: utf-8 -*-
"""
given a word and visualize near words
original source code is https://github.com/nishio/mycorpus/blob/master/vis.py
"""
import word2vec_boostpython as w2v
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.font_manager

class visWord2Vec:
    def __init__(self, filename = 'vectors.bin'):
        font = matplotlib.font_manager.FontProperties(fname='./ipag.ttc')
        FONT_SIZE = 20
        self.TEXT_KW = dict(fontsize=FONT_SIZE, fontweight='bold', fontproperties=font)

        print 'loading'
        self.data = w2v.load(filename)
        print 'loaded'

    def plot(self, query, nbest = 15):
        if ', ' not in query:
            words = [query] + w2v.search(self.data, query)[:nbest]
        else:
            words = query.split(', ')
            print ', '.join(words)
        mat = w2v.get_vectors(self.data)
        word_indexes = [w2v.get_word_index(self.data, w) for w in words]
        if word_indexes == [-1]:
            print 'not in vocabulary'
            return

        # do PCA
        X = mat[word_indexes]
        pca = PCA(n_components=2)
        pca.fit(X)
        print pca.explained_variance_ratio_
        X = pca.transform(X)
        xs = X[:, 0]
        ys = X[:, 1]

        # draw
        plt.figure(figsize=(12,8))
        plt.scatter(xs, ys, marker = 'o')
        for i, w in enumerate(words):
            plt.annotate(
                w.decode('utf-8', 'ignore'),
                xy = (xs[i], ys[i]), xytext = (3, 3),
                textcoords = 'offset points', ha = 'left', va = 'top',
                **self.TEXT_KW)

        plt.show()

In [ ]:
import visword2vec
vis = visword2vec.visWord2Vec("vectors.bin")
vis_phrase = visword2vec.visWord2Vec("vectors-phrase.bin")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords

train_set = candidates_df['textR']    #Documents
test_set = ['dominance','authorization','potency',' intensity','force','control','dominat','strengthen','authorize','evil','immoral','obey','character','moral','crimes','punishment','prosecute','multiculturism','values','military','federal','government','taxes','violence','amnesty','illegally','bad','disaster','stop','trouble','dishorned','return','proliferate,','elite','illegal','stolen','criminally','criminal','jail','crooked','charges','false','disgrace','lawsuits','corrupt','guilty','fraud','racketeering','judgement','institutions','dependence','legitimize','repressive','brutal','agressive','kill','exploit','national','greed','excess','corruption','evil','spending','runaway','indictment','unwise','bankrupt','consequence','torture','capture','crush','win','poppy','drug','heroin','failed','fault','defeated','withdrawal','league','execut','sanctions','rogue','regime','torture','sacrifice','right','wrong','adversaries','military','control','states','local','parents','lower','corporations','rich','criminal','justice','penalty','law','order','police','second','amendment','constitution','uphold','seige','protecting','defending','defend','status','founding','fathers','again','honored','back','punished','punish','strong','borders','wall','secure'' private ','powerful','tough','tougher','top','fallen','warheads','renegotiate','deals','ideals','isolate','isolation','trust','verify','oversight','enforce','proud','parents','powerful','wealthy','money','negotiate','leaderships','leaders','tougher','right','repeal','strongly','failure','responsibility','win','victory','surge','stability','respect','restraint','solidarity','rewarded','influence','power','stricter','fundamental','belief','goodness','paternal','dependent','under','eliminate','freeze','budget','victory','honor','back','holding','responsibly','bolster','toughest','unwarrented','acceptable','behavior','violation','sieze','respects','illegal','objectionable','secure','invasion','investing','influence','offshore','overseas','oversight','overtime','inside','accountability','transparency']

# sf_list manually creatd by reviewing 2 debate texts 'dominance','authorization','potency','intensity','force','control']   #Query
stopWords = stopwords.words('english')

vectorizer = CountVectorizer(stop_words = stopWords)
#print vectorizer
transformer = TfidfTransformer()
#print transformer

trainVectorizerArray = vectorizer.fit_transform(train_set).toarray()
testVectorizerArray = vectorizer.transform(test_set).toarray()
#print 'Fit Vectorizer to train set', trainVectorizerArray
#print 'Transform Vectorizer to test set', testVectorizerArray

transformer.fit(trainVectorizerArray)
#print
#print transformer.transform(trainVectorizerArray).toarray()

transformer.fit(testVectorizerArray)
#print 
tfidf = transformer.transform(testVectorizerArray)
print tfidf.todense()
print vectorizer.vocabulary_